In [7]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [9]:
df = pd.read_csv('hw-data - q4.csv')

In [37]:
# pipe = Pipeline([
#     ('OH', OneHotEncoder()),
#     ('LM', LinearRegression()),
# ])
ox = OneHotEncoder().fit_transform([[v] for v in df["cond"]])
model = LinearRegression()
model.fit(ox, df["g1"])

LinearRegression()

In [38]:
model.predict(ox)

array([-0.70588935, -0.70588935, -0.70588935, -0.70588935, -0.70588935,
       -0.70588935,  1.91787316,  1.91787316,  1.91787316,  1.91787316,
       -0.70588935, -0.70588935, -0.70588935, -0.70588935,  1.91787316,
        1.91787316,  1.91787316,  1.91787316,  1.91787316])

In [40]:
ys = df["g1"].to_numpy()
pred_ys = model.predict(ox)
r2 = r2_score(ys, pred_ys)

In [44]:
mse = np.sum((pred_ys - ys) ** 2, axis=0) / float(ox.shape[0] - ox.shape[1])
mse

2.842430974557567

In [50]:
model.coef_

array([-1.31188125,  1.31188125])

In [66]:
std = stats.tstd(pred_ys)
se = (std / np.sqrt(pred_ys.shape))[0]
t = model.coef_[0] / se
p = 2 * (1 - stats.t.cdf(np.abs(t), ys.shape[0] - ox.shape[1]))
t, p

(-4.248529157249601, 0.0005418363973797025)

In [67]:
r2_score(ys, pred_ys)

0.4029270911398082

In [39]:
# adapted from
# https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression
def model_stats(model, xs, ys):
    # if not isinstance(xs, np.ndarray): xs = np.array(xs)
    # if not isinstance(ys, np.ndarray): ys = np.array(ys)
    pred_ys = model.predict(xs)
    sse = np.sum((pred_ys - ys) ** 2, axis=0) / float(xs.shape[0] - xs.shape[1])
    if isinstance(sse, float): sse = np.array([sse])
    se = np.array([
        np.sqrt(np.diagonal(sse[i] * np.linalg.inv(np.dot(xs.T, xs))))
    for i in range(sse.shape[0])], dtype=float)

    t = model.coef_ / se
    p = 2 * (1 - stats.t.cdf(np.abs(t), ys.shape[0] - xs.shape[1]))
    r2 = r2_score(ys, pred_ys)
    return t, p, r2

model_stats(model, ox, df["g1"])

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional